In [17]:
#checking 100 iter from cmd line train
import os
import torch
os.chdir('/mnt/disks/uberduck-experiments-v0/mellotron')

from hparams import create_hparams
from model import Tacotron2
from text import cmudict, text_to_sequence
from train import warm_start_model
from loss_function import Tacotron2Loss
#import waveglow.glow
#from waveglow.denoiser import Denoiser
import waveglow
import sys
import IPython.display as ipd
from train import prepare_dataloaders
import time 
sys.path.append('waveglow/')
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )

def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model


#waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
#waveglow_2 = torch.load(waveglow_path)['model'].cuda().eval()
#denoiser = Denoiser(waveglow_2).cuda().eval()

hparams = create_hparams(None)
hparams.n_speakers = 3
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'

#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/lj_929_2/checkpoint_0'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/em_930/checkpoint_1500'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/em_lt_930/checkpoint_3500'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3_lj_930/checkpoint_3500'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3_lj_930/checkpoint_2500'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3_lt_1001/checkpoint_2500'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3_scratch_1001/checkpoint_1500'
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
learning_rate = 1e-3#
hparams.use_saved_learning_rate = True
torch.manual_seed(hparams.seed)

model = load_model(hparams)
criterion = Tacotron2Loss()
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)


Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'


In [36]:
ignore_layers

['speaker_embedding.weight']

In [38]:
model = Tacotron2(hparams).cuda()
checkpoint = torch.load(hparams.checkpoint_path, map_location='cpu')
model_dict = checkpoint['state_dict']
ignore_layers = hparams.ignore_layers
if "state_dict" in checkpoint.keys():
    model_dict = checkpoint["state_dict"]
    if ignore_layers:
        model_dict = {k: v for k, v in model_dict.items()
                      if k not in ignore_layers}
        dummy_dict = model.state_dict()
        dummy_dict.update(model_dict)
        model_dict = dummy_dict
    model.load_state_dict(model_dict)
